In [3]:
from langchain.document_loaders import WebBaseLoader

# List of URLs for stock data
urls = [
    "https://finance.yahoo.com/quote/AAPL",  # Yahoo Finance - Apple
    "https://www.investing.com/equities/apple-computer-inc",  # Investing.com - Apple
    "https://www.moneycontrol.com/india/stockpricequote/technology/infosys/IT"  # Moneycontrol - Infosys
]

# Load documents from all URLs
loader = WebBaseLoader(urls)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")


Number of documents loaded: 3


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter()

# Split documents into chunks
doc_chunks = text_splitter.split_documents(docs)
print(f"Number of document chunks: {len(doc_chunks)}")


Number of document chunks: 41


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

import os
from getpass import getpass

# get your free access token from HuggingFace and paste it here
HF_token = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

embeddings = HuggingFaceEmbeddings(
    api_key=HF_token, model_name="BAAI/bge-base-en-v1.5"
)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_1172\3085998862.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


ValidationError: 1 validation error for HuggingFaceEmbeddings
api_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [ ]:
doc_embeddings = embeddings.embed_documents([chunk.page_content for chunk in doc_chunks])